In [2]:
!pip install python-igraph

    100% |████████████████████████████████| 378kB 2.3MB/s eta 0:00:01
  Stored in directory: /home/kurmukov/.cache/pip/wheels/41/d6/02/34eebae97e25f5b87d60f4c0687e00523e3f244fa41bc3f4a7
Successfully built python-igraph


In [1]:
import numpy as np
from igraph import Graph
from igraph import ADJ_MAX
from scipy.sparse import coo_matrix
from sys import argv
from tqdm import tqdm


In [2]:
def compute_partition(adjacency):
    try:
        g = Graph.Weighted_Adjacency(adjacency.tolist(),
                                     mode=ADJ_MAX,
                                     attr='weight')
        levels = g.community_multilevel(
            weights='weight',
            return_levels=True)
        levels = np.array([level.membership for level in levels]).astype(int)
        labels = np.array(levels[-1])
    except:
        labels = np.array([-1])  # dangerous, what if i have a single node in a graph?
    return labels

In [3]:
def load_concon(path):
    from scipy.sparse import coo_matrix
    data = np.load(path)
    sparse_data = coo_matrix((data['data'], (data['row'], data['col'])))
    adj = sparse_data.todense()
    adjacency_balanced = (adj + adj.T)/2
    np.fill_diagonal(adjacency_balanced, 0)
    return adjacency_balanced

In [4]:
path = '/data01/ayagoz/sparse_32_concon_HCP/ConCon_resolution/10/100307.npz'

adj = load_concon(path)

In [5]:
g = Graph.Weighted_Adjacency(adj.tolist(),
                             mode=ADJ_MAX,
                             attr='weight')

In [6]:
levels = g.community_multilevel(
            weights='weight',
            return_levels=True)

In [7]:
levels

In [8]:
levels = np.array([level.membership for level in levels]).astype(int)

In [10]:
levels.shape

(2, 20484)

In [11]:
levels[0]

array([ 6,  3,  4, ..., 13, 13, 13])

In [12]:
np.unique(levels[0], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
 array([   1, 1562, 1224, 1356, 2048,  229, 1776, 4099,  865,    1, 1660,
         750,  580, 3197,  892,  244]))

In [13]:
np.unique(levels[1], return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([   1, 3922, 4498, 1776,    1, 1660, 4099, 1330, 3197]))

In [15]:
from sklearn.metrics import adjusted_mutual_info_score

In [18]:
color = levels[0]

In [19]:
adjusted_mutual_info_score(color[:10242], color[10242:])

0.2703792240138568

In [21]:
aver = np.load('/data01/ayagoz/sparse_32_concon_HCP/parcellations/desikan_aparc_average_1113.npy',
               allow_pickle=True)

In [22]:
mask = aver != -1
adj_masked = adj[mask, :][:, mask]

In [23]:
adj_masked.shape

(18869, 18869)

In [30]:
g = Graph.Weighted_Adjacency(adj_masked.tolist(),
                             mode=ADJ_MAX,
                             attr='weight')

levels_new = g.community_multilevel(
            weights='weight',
            return_levels=True)

In [31]:
levels_new = np.array([level.membership for level in levels_new]).astype(int)

In [32]:
levels_new

array([[ 9,  5,  7, ..., 14, 14, 14],
       [ 3,  1,  2, ...,  6,  6,  6]])

In [33]:
labels_temp = levels_new[-1]
labels_new = np.zeros(20484) - 1
labels_new[mask] = labels_temp

In [34]:
adjusted_mutual_info_score(levels[-1], labels_new)

0.637412873493857

In [35]:
medial_labels = levels[-1][aver==-1]

In [37]:
np.unique(medial_labels, return_counts=True)

(array([1, 2, 3, 5, 6, 7, 8]), array([359, 503, 226,  12,  93,  44, 378]))

In [68]:
def load_concon(path, labels_to_drop=None):
    '''
    Load ConCon ajancency matrix
    
    Parameters
    -------
    
    path - str,
     path to concon *.npz file
     
    labels_to_drop - ndarray,
     default None, drops rows/columns which
     corresponds to label "-1" (corpus collosum, cerebellum)
     
    Return
    -------
    
    adjacency matrix
    '''

    data = np.load(path)
    sparse_data = coo_matrix((data['data'], (data['row'], data['col'])))
    adj = sparse_data.todense()
    adjacency_balanced = (adj + adj.T) / 2
    np.fill_diagonal(adjacency_balanced, 0)

    if labels_to_drop is not None:
        mask = labels_to_drop != -1
        adjacency_balanced = adjacency_balanced[mask, :][:, mask]
    return adjacency_balanced

def compute_partition(adj):
    '''
    Computes partition of the graph using Louvain algorithm
    
    Parameters
    -------
    
    graph - igraph Graph
    
    Returns
    -------
    
    labels - ndarray
    '''
    try:
        graph = Graph.Weighted_Adjacency(adj.tolist(), mode=ADJ_MAX, attr='weight')
        levels = graph.community_multilevel(weights='weight', return_levels=True)
        levels = np.array([level.membership for level in levels]).astype(int)
        labels = np.array(levels[-1])
    except BaseException as e:
        print(e)
        labels = np.array([-1]) 
    return labels

def generate_subgraphs(adjacency, partition, min_subgraph_size=2):
    '''
    Generates subgraphs based on partition
    
    Parameters
    -------
    
    adjacency - ndarray,
     adjacecy matrix of a graph
     
    partition - ndarray,
     clustering labels of graph's nodes
     
    min_graph_size - int,
     minimal size subgraph
     (used to pass isolated nodes)
     
    Returns
    -------
    
    adjacency_list - list,
     list of adjacency matrices of generated subgraphs
     
    colors - ndarray,
     array of unique cluster labels,
     excluding rare clusters (len(cluster)<min_graph_size)
     
    rare_colors - ndarray,
     array of unique cluster labels,
     of rare clusters (len(cluster)<min_graph_size)

    '''
    colors, freq = np.unique(partition, return_counts=True)
    rare_colors = colors[freq < min_subgraph_size]
    colors = colors[freq >= min_subgraph_size]
    
    adjacencies_list = [adjacency[partition==color, :][:, partition==color]
                     for color in colors]
    
    return adjacencies_list, colors, rare_colors


def cluster_subgraphs(adjacency, partition, min_subgraph_size=10, min_subgraph_to_split=30):
    '''
    Cluster subgraphs of a graph
    
    Parameters
    --------
    
    adjacency - ndarray,
     adjacecy matrix of a graph
     
    partition - ndarray,
     clustering labels of graph's nodes
     
    min_graph_size - int,
     minimal size subgraph
     (used to pass isolated nodes)
     
    min_subgraph_to_split - int,
     minimal size subgraph to split
     (used to restrict small subgraphs to be further clustered)
     
    Returns
    -------
    
    partition_at_level - ndarray,
     hierarchical graph partitioning
    '''
    adjacencies_list, colors, rare_colors = generate_subgraphs(adjacency, partition, min_subgraph_size)
    partition_at_level = np.zeros_like(partition)

    if rare_colors.size != 0:
        for c in rare_colors:
            partition_at_level[np.where(partition == c)] = -1

    max_label = 0

    for adj, c in zip(adjacencies_list, colors):
        subgraph_size = adj.shape[0]
        if subgraph_size < min_subgraph_to_split:
            labels = np.zeros(subgraph_size)
        else:
            labels = compute_partition(adj)
        # to be consistent in labeling accross different subgraphs
        partition_at_level[np.where(partition == c)[0]] = labels + max_label
        max_label += labels.max() + 1

    return partition_at_level

In [69]:
aver = np.load('/data01/ayagoz/sparse_32_concon_HCP/parcellations/desikan_aparc_average_1113.npy',
               allow_pickle=True)
path = '/data01/ayagoz/sparse_32_concon_HCP/ConCon_resolution/10/100307.npz'
adj = load_concon(path, labels_to_drop=aver)
partition_level_1 = compute_partition(adj)

In [70]:
np.unique(partition_level_1, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([   1, 4397, 2613, 1585, 4071,    1, 2631, 3570]))

In [71]:
partition_level_2 = cluster_subgraphs(adj, partition_level_1, 5, 200)

In [72]:
np.unique(partition_level_2, return_counts=True)

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]),
 array([   2,  840,  445,  524,  676, 1067,  845,  311,  664,  478,  547,
         377,  236,  175,  361,  340,  432,  277,  426, 1428,  459,  801,
         957,  199,  523,  446,  394,  838,  231,  436,  664,  892,  772,
         806]))

In [73]:
partition_level_3 = cluster_subgraphs(adj, partition_level_2, 5, 360)

In [74]:
np.unique(partition_level_3, return_counts=True)

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
        33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
        50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
        67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,
        84, 85, 86, 87, 88, 89, 90, 91, 92]),
 array([  2, 147, 190, 238, 265,  89,  91, 148, 117, 169, 174, 181,  71,
        279, 326, 223, 350, 410,  84, 281, 261, 303, 311, 160,  76, 236,
        192, 188, 146, 144, 297, 250, 150, 136,  91, 236, 175, 186, 175,
        340,  43, 173, 216, 277, 102, 167,  38, 119, 261, 362, 186, 619,
        109, 144, 206, 139, 307, 355, 440, 371, 146, 199, 220, 303, 140,
        125, 181, 146, 134, 114, 351, 306, 181, 231, 125,  97, 138,  76,
        169, 130,  73, 127, 165, 236, 179, 157, 320, 277, 151, 344, 247,
        198, 243, 118]))

In [ ]:
average_desikan = np.load('/data01/ayagoz/sparse_32_concon_HCP/parcellations/desikan_aparc_average_1113.npy',
               allow_pickle=True)
path = '/data01/ayagoz/sparse_32_concon_HCP/ConCon_resolution/10/100307.npz'
adj = load_concon(path, labels_to_drop=average_desikan)

partition_level_1 = compute_partition(adj)
partition_level_2 = cluster_subgraphs(adj, partition_level_1, 5, 200)
partition_level_3 = cluster_subgraphs(adj, partition_level_2, 5, 360)

np.save()

In [41]:
graph = Graph.Weighted_Adjacency([[10]], mode=ADJ_MAX, attr='weight')

In [42]:
levels = graph.community_multilevel(weights='weight', return_levels=True)

In [43]:
levels = np.array([level.membership for level in levels]).astype(int)

In [44]:
levels

array([], dtype=int64)

In [ ]:
import os
import numpy as np
from scipy.sparse import coo_matrix
from tqdm import tqdm
from sys import argv
from joblib import Parallel, delayed

def load_concon(path, labels_to_drop=None):
    '''
    Load ConCon ajancency matrix
    
    Parameters
    -------
    
    path - str,
     path to concon *.npz file
     
    labels_to_drop - ndarray,
     default None, drops rows/columns which
     corresponds to label "-1" (corpus collosum, cerebellum)
     
    Return
    -------
    
    adjacency matrix
    '''

    data = np.load(path)
    sparse_data = coo_matrix((data['data'], (data['row'], data['col'])))
    adj = sparse_data.todense()
    adjacency_balanced = (adj + adj.T) / 2
    np.fill_diagonal(adjacency_balanced, 0)

    if labels_to_drop is not None:
        mask = labels_to_drop != -1
        adjacency_balanced = adjacency_balanced[mask, :][:, mask]
    return adjacency_balanced

def compute_partition(adj):
    '''
    Computes partition of the graph using Louvain algorithm
    
    Parameters
    -------
    
    graph - igraph Graph
    
    Returns
    -------
    
    labels - ndarray
    '''
    try:
        graph = Graph.Weighted_Adjacency(adj.tolist(), mode=ADJ_MAX, attr='weight')
        levels = graph.community_multilevel(weights='weight', return_levels=True)
        levels = np.array([level.membership for level in levels]).astype(int)
        labels = np.array(levels[-1])
    except BaseException as e:
        print(e)
        labels = np.array([-1]) 
    return labels

def generate_subgraphs(adjacency, partition, min_subgraph_size=2):
    '''
    Generates subgraphs based on partition
    
    Parameters
    -------
    
    adjacency - ndarray,
     adjacecy matrix of a graph
     
    partition - ndarray,
     clustering labels of graph's nodes
     
    min_graph_size - int,
     minimal size subgraph
     (used to pass isolated nodes)
     
    Returns
    -------
    
    adjacency_list - list,
     list of adjacency matrices of generated subgraphs
     
    colors - ndarray,
     array of unique cluster labels,
     excluding rare clusters (len(cluster)<min_graph_size)
     
    rare_colors - ndarray,
     array of unique cluster labels,
     of rare clusters (len(cluster)<min_graph_size)

    '''
    colors, freq = np.unique(partition, return_counts=True)
    rare_colors = colors[freq < min_subgraph_size]
    colors = colors[freq >= min_subgraph_size]
    
    adjacencies_list = [adjacency[partition==color, :][:, partition==color]
                     for color in colors]
    
    return adjacencies_list, colors, rare_colors


def cluster_subgraphs(adjacency, partition, min_subgraph_size=10, min_subgraph_to_split=30):
    '''
    Cluster subgraphs of a graph
    
    Parameters
    --------
    
    adjacency - ndarray,
     adjacecy matrix of a graph
     
    partition - ndarray,
     clustering labels of graph's nodes
     
    min_graph_size - int,
     minimal size subgraph
     (used to pass isolated nodes)
     
    min_subgraph_to_split - int,
     minimal size subgraph to split
     (used to restrict small subgraphs to be further clustered)
     
    Returns
    -------
    
    partition_at_level - ndarray,
     hierarchical graph partitioning
    '''
    adjacencies_list, colors, rare_colors = generate_subgraphs(adjacency, partition, min_subgraph_size)
    partition_at_level = np.zeros_like(partition)

    if rare_colors.size != 0:
        for c in rare_colors:
            partition_at_level[np.where(partition == c)] = -1

    max_label = 0

    for adj, c in zip(adjacencies_list, colors):
        subgraph_size = adj.shape[0]
        if subgraph_size < min_subgraph_to_split:
            labels = np.zeros(subgraph_size)
        else:
            labels = compute_partition(adj)
        # to be consistent in labeling accross different subgraphs
        partition_at_level[np.where(partition == c)[0]] = labels + max_label
        max_label += labels.max() + 1

    return partition_at_level

def get_partitions_at_levels(subject_id, labels_to_drop):
    
    for sparsity in range(10, 101, 10):
        try:
            path = f'/data01/ayagoz/sparse_32_concon_HCP/ConCon_resolution/{sparsity}/{subject_id}.npz'
            adj = load_concon(path, labels_to_drop=labels_to_drop)
            partition_level_1 = compute_partition(adj)
            partition_level_2 = cluster_subgraphs(adj, partition_level_1, 5, 200)
            partition_level_3 = cluster_subgraphs(adj, partition_level_2, 5, 360)
            parcellation_path = '/data01/ayagoz/sparse_32_concon_HCP/parcellations'
            np.save(f'{parcellation_path}/connectivity_parcellation_level1/{sparsity}/{subject_id}.npy',
                   partition_level_1)
            np.save(f'{parcellation_path}/connectivity_parcellation_level2/{sparsity}/{subject_id}.npy',
                   partition_level_2)
            np.save(f'{parcellation_path}/connectivity_parcellation_level3/{sparsity}/{subject_id}.npy',
                   partition_level_3)


if __name__=="__main__":

    average_desikan = np.load('/data01/ayagoz/sparse_32_concon_HCP/parcellations/desikan_aparc_average_1113.npy',
                   allow_pickle=True)
    

    path = '/data01/ayagoz/sparse_32_concon_HCP/ConCon_resolution/10/100307.npz'
    adj = load_concon(path, labels_to_drop=average_desikan)

    partition_level_1 = compute_partition(adj)
    partition_level_2 = cluster_subgraphs(adj, partition_level_1, 5, 200)
    partition_level_3 = cluster_subgraphs(adj, partition_level_2, 5, 360)

    np.save()

In [2]:
import os
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from scipy.sparse import coo_matrix
import numpy as np
from scipy.sparse import coo_matrix

def load_concon(path, labels_to_drop=None):
    '''
    Load ConCon ajancency matrix
    
    Parameters
    -------
    
    path - str,
     path to concon *.npz file
     
    labels_to_drop - ndarray,
     default None, drops rows/columns which
     corresponds to label "-1" (corpus collosum, cerebellum)
     
    Return
    -------
    
    adjacency matrix
    '''

    data = np.load(path)
    sparse_data = coo_matrix((data['data'], (data['row'], data['col'])))
    adj = sparse_data.todense()
    adjacency_balanced = (adj + adj.T) / 2
    np.fill_diagonal(adjacency_balanced, 0)

    if labels_to_drop is not None:
        mask = labels_to_drop != -1
        adjacency_balanced = adjacency_balanced[mask, :][:, mask]
    return adjacency_balanced

def squeeze_matrix(matrix,
                   labels,
                   drop_minus_1 = False,
                   fill_diag_zero=True,
                   return_transform=False,):
    '''
    Apply parcellation to adjacency matrix
    
    Parameters
    ----
    
    matrix - ndarray,
     graph adjacency matrix M x M
     
    labels - ndarray,
     parcellation labels 1 x M with N unique values
     (cluster/parcels labels)
     
    drop_minus_1 - bool,
     default True, drops rows/columns which
     corresponds to label "-1"
    
    fill_diag_zero - bool,
     default, True, fills diagonal of the
     returned adjacency matrix with zeros    
              
    return_transform - bool,
     default False
    '''
    if drop_minus_1:
        mask = labels != -1
        labels = labels[mask]
        matrix = matrix[mask, :][:, mask]

    input_size = matrix.shape[0]
    output_size = np.unique(labels).shape[0]
    
    d = dict(zip(np.unique(labels), np.arange(output_size)))
    _labels = list(map(lambda x: d[x], labels))
    transform = np.zeros((input_size, output_size))
    
    for row, col in enumerate(_labels):
        transform[row, col] = 1
    
    squeezed = transform.T.dot(matrix.dot(transform))
    squeezed = (squeezed + squeezed.T)/2
    
    if fill_diag_zero:
        np.fill_diagonal(squeezed, 0)
    
    if return_transform:
        return np.array(squeezed), transform
    
    return np.array(squeezed)

def apply_parcellation(subject):
    '''
    Applies parcellation for all sparsity levels
    
    Parameters
    ------
    
    subject - int,
     subject id
    '''
    average_desikan = np.load('/data01/ayagoz/sparse_32_concon_HCP/parcellations/desikan_aparc_average_1113.npy',
                   allow_pickle=True)
    source = '/data01/ayagoz/sparse_32_concon_HCP/connectomes/ConCon_resolution'
    parcellation = '/data01/ayagoz/sparse_32_concon_HCP/parcellations/ensemble_parcellation/connectivity_parcellation_level'
    target_folder = '/data01/ayagoz/sparse_32_concon_HCP/connectomes/individual_louvain_parcellation'

    for sparsity in range(10, 101, 10):
        try:
            adj = load_concon(f'{source}/{sparsity}/{subject}.npz', labels_to_drop=average_desikan)
#             print(f'{source}/{sparsity}/{subject}.npz')
            for i in [1,2,3]:
                labels_parcellation = np.load(f'{parcellation}{i}/{sparsity}/ensemble_{i}_{sparsity}.npy')
#                 print(f'{parcellation}{i}/{sparsity}/ensemble_{i}_{sparsity}.npy')
                adj_parcellation = squeeze_matrix(adj, labels_parcellation)
                np.save(f'{target_folder}/level{i}/{sparsity}/{subject}.npy', adj_parcellation)
#                 print(f'{target_folder}/level{i}/{sparsity}/{subject}.npy')
        except BaseException as e:
            print(e, subject, sparsity, plevel)


if __name__ == "__main__":

    source = '/data01/ayagoz/sparse_32_concon_HCP/connectomes/ConCon_resolution'
    all_subjects = [uid.split('.')[0] for uid in os.listdir(f'{source}/10')]
    Parallel(n_jobs=25)(delayed(apply_parcellation)(subject) for subject in tqdm(all_subjects))
    
    

In [3]:
apply_parcellation('100307')

In [17]:
a= np.load('/data01/ayagoz/sparse_32_concon_HCP/connectomes/individual_louvain_parcellation/level3/10/100307.npy')

In [18]:
a.shape

(91, 91)

In [19]:
source = '/data01/ayagoz/sparse_32_concon_HCP/connectomes/ConCon_resolution'
all_subjects = [uid.split('.')[0] for uid in os.listdir(f'{source}/10')]

In [20]:
all_subjects

['180836',
 '212217',
 '173536',
 '175035',
 '175439',
 '200614',
 '295146',
 '104416',
 '157336',
 '151627',
 '111716',
 '162935',
 '151223',
 '187547',
 '160729',
 '119833',
 '198350',
 '119732',
 '134425',
 '154936',
 '209228',
 '160931',
 '285446',
 '143325',
 '111312',
 '148840',
 '140420',
 '110007',
 '108525',
 '211215',
 '159946',
 '156435',
 '103515',
 '214019',
 '192035',
 '176744',
 '168745',
 '170934',
 '245333',
 '208226',
 '194746',
 '174841',
 '227432',
 '160123',
 '153631',
 '181636',
 '203418',
 '141826',
 '100408',
 '173435',
 '117930',
 '154835',
 '138231',
 '223929',
 '105014',
 '173738',
 '209127',
 '192843',
 '322224',
 '166438',
 '199453',
 '107018',
 '268850',
 '187345',
 '212419',
 '167036',
 '135528',
 '268749',
 '130821',
 '164939',
 '165032',
 '202719',
 '285345',
 '165638',
 '201414',
 '246133',
 '201111',
 '123420',
 '161327',
 '150726',
 '107725',
 '154431',
 '127630',
 '118023',
 '162329',
 '156536',
 '182739',
 '112314',
 '280739',
 '186141',
 '159441',